In [ ]:
import pcraster as pcr
pcr.setrandomseed(8)
buildgMap = pcr.readmap("mapalgebra/buildg.map")
firestatMap = pcr.readmap("mapalgebra/firestat.map")
iswaterMap = pcr.readmap("mapalgebra/iswater.map")
phreaticMap = pcr.readmap("mapalgebra/phreatic.map")
rainstorMap = pcr.readmap("mapalgebra/rainstor.map")
roadsMap = pcr.readmap("mapalgebra/roads.map")
topoMap = pcr.readmap("mapalgebra/topo.map")
waterMap = pcr.readmap("mapalgebra/water.map")
dumpMap = pcr.readmap("mapalgebra/dump.map")
isroadMap = pcr.readmap("mapalgebra/isroad.map")
loggingMap = pcr.readmap("mapalgebra/logging.map")
pointsMap = pcr.readmap("mapalgebra/points.map")
rainyearMap = pcr.readmap("mapalgebra/rainyear.map")
soilsMap = pcr.readmap("mapalgebra/soils.map")
treesMap = pcr.readmap("mapalgebra/trees.map")
wellsMap = pcr.readmap("mapalgebra/wells.map")

In this exercise you will use the dataset introduced in the visualisation section to build a simple spatial hydrological model, including rainfall, snowfall, snowmelt, and runoff.

Open dynamicTimeSeries.py created in the previous exercises, save it under a new name, temp.py. It reads the precipitation timeseries from disk. Add a statement that reads the temperature timeseries temp.tss from disk and assigns the temperature values (degrees celcius) to the map variable temperatureObserved. Write the results to disk, run the model, and have a look at the output.

Question 11: Which function did you use to read the temperature from disk?

<blockquote>
<div><ol class="loweralpha simple">
<li><p><code class="docutils literal notranslate"><span class="pre">open</span></code></p></li>
<li><p><code class="docutils literal notranslate"><span class="pre">timeinputscalar</span></code></p></li>
<li><p><code class="docutils literal notranslate"><span class="pre">timeinputnominal</span></code></p></li>
<li><p><code class="docutils literal notranslate"><span class="pre">timeinput</span></code></p></li>
<li><p><code class="docutils literal notranslate"><span class="pre">report</span></code></p></li>
</ol>
</div></blockquote>

The temperature timeseries contains the temperature observed at a meteostation that is close by. However, temperature will be spatially variable, as it depends on the surface elevation. The temperature ti at a grid cell i can be calculated as:

ti = tobs  - l (ei  - eobs)

In the equation, tobs is the temperature observed at the meteostation, ei is the elevation at a grid cell, eobs is the elevation at the meteo station (here: 2058.1 m) and l is the temperature lapse rate (here: 0.005).

Add statements to your script temp.py to calculate a map (call it temperatureCorrection) containing the term l (ei  - eobs). Write the map to disk and check the results.

Question 12: Where did you add the statements to calculate temperatureCorrection?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>In the <code class="docutils literal notranslate"><span class="pre">__init__</span></code>.</p></li>
<li><p>In the <code class="docutils literal notranslate"><span class="pre">initial</span></code>.</p></li>
<li><p>In the <code class="docutils literal notranslate"><span class="pre">dynamic</span></code>.</p></li>
<li><p>In the <code class="docutils literal notranslate"><span class="pre">initial</span></code> and in the <code class="docutils literal notranslate"><span class="pre">dynamic</span></code></p></li>
</ol>
</div></blockquote>

Question 13: How much colder is it at the highest point in the area compared to the temperature at the meteostation (i.e., at 2058.1 m)?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>About 2 degrees.</p></li>
<li><p>About 12 degrees.</p></li>
<li><p>About 7 degrees.</p></li>
<li><p>There is no difference.</p></li>
</ol>
</div></blockquote>

As a next step, calculate and write to disk a time series of maps containing ti. Save the script (temp.py), run and display the new output.

We assume that precipitation falls as snow when tiis below zero. Add statements to calculate and store a time series of maps (use the variable name freezing) that is True at a cell if the temperature is below zero degrees and False elsewhere. Use a comparison operator. Run the script and check the results.

Create two time series of maps containing respectively snowfall (m/day, variable name snowFall) and rainfall (m/day, variable name rainFall). You will need the operations ifthenelse and a Boolean operator. Write them to disk and display the results.

Question 14: Which comparison operator did you use, and where?

<blockquote>
<div><ol class="loweralpha simple">
<li><p><code class="docutils literal notranslate"><span class="pre">==</span></code>, in the <code class="docutils literal notranslate"><span class="pre">dynamic</span></code></p></li>
<li><p><code class="docutils literal notranslate"><span class="pre">or</span></code>, in the <code class="docutils literal notranslate"><span class="pre">dynamic</span></code></p></li>
<li><p><code class="docutils literal notranslate"><span class="pre">while</span></code>, in the <code class="docutils literal notranslate"><span class="pre">initial</span></code></p></li>
<li><p><code class="docutils literal notranslate"><span class="pre">&gt;</span></code> or <code class="docutils literal notranslate"><span class="pre">&lt;</span></code> or <code class="docutils literal notranslate"><span class="pre">&gt;=</span></code> or <code class="docutils literal notranslate"><span class="pre">&lt;=</span></code>, in the <code class="docutils literal notranslate"><span class="pre">dynamic</span></code></p></li>
</ol>
</div></blockquote>

To simulate snow melt in each cell, a snow store is used containing a snow pack ai(t) (m water equivalent) that changes over time:

ai(t) = ai(t-1) +  si(t) - bi(t)

In the equation above, (t) refers to the current time step and (t-1) to the previous time step (day). The variable si(t) is snowfall and bi(t) is snowmelt. Actual snowmelt bi(t) is modelled as:

bi(t) = min(ai(t), bp,i(t))

with, bp,i(t), potential snowmelt. Potential snowmelt is:

bp,i(t) = mti, for ti > 0

bp,i(t) = 0, for ti <= 0

with m, a degree day factor (here: 0.01), and ti , the corrected termperature (see above).

Add this snow store in a stepwise manner to the model, in the following way. Let’s first assume there is no snow melt. Open temp.py and save it as snow.py. Add the variable snow representing the snow store ai, assuming there is no snow at the start of the model run (as it is end of summer), and increasing the snowstore with snowfall in each time step.

Question 15: Without snow melt (as was assumed here), what is the maximum snowpack thickness (m water equivalent) found in the area at the end of the model run? Idem, the minimum thickness?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>Maximum thickness: 0.11 m, minimum: 0.01 m</p></li>
<li><p>Maximum thickness: 1.12 m, minimum: 0.15 m</p></li>
<li><p>Maximum thickness: 0.85 m, minimum: 0.00 m</p></li>
<li><p>Maximum thickness: 1.96 m, minimum: 0.62 m</p></li>
<li><p>Maximum thickness: 0.57 m, minimum: 0.01 m</p></li>
</ol>
</div></blockquote>

Add snowmelt to the model. First, calculate a variable potentialMelt, containing bp,i(t). Write to disk and check the results. Second, calculate actual snowmelt bi(t). Check the results.

Question 16: How did you calculate actual snowmelt?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>It is equal to the potential snow melt in this case.</p></li>
<li><p>It is a fraction of the potential melt.</p></li>
<li><p>It depends on the potential melt and the actual snow thickness.</p></li>
<li><p>It is constant, so it needs to be calculated in the <code class="docutils literal notranslate"><span class="pre">initial</span></code>.</p></li>
</ol>
</div></blockquote>

And as the third step, update the snow store by subtracting actual snowmelt from the store. Write the snow store to disk (directly after subtracting actual snowmelt). Save the model under the same name snow.py.

Question 17: From what time step on is the main valley in the top-right corner snow-free, in spring? Idem, for the valley in the top-left corner?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>Top-right corner: day 128. Top-left corner: day 130.</p></li>
<li><p>Top-right corner: day 110. Top-left corner: day 110.</p></li>
<li><p>Top-right corner: day 112. Top-left corner: day 140.</p></li>
<li><p>Top-right corner: day 92. Top-left corner: day 118.</p></li>
</ol>
</div></blockquote>

Runoff generated in each cell (m/day) is the sum of the rainfall in that cell and the snowmelt. Add a statement to snow.py calculating a time series of maps containing the total amount of runoff generated. Write results to disk and evaluate the results.

Question 18: In what time of the year is the largest amount of runoff generated?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>Melting snow generates a large amount of runoff around time step 70.</p></li>
<li><p>Melting snow generates a large amount of runoff around time step 90.</p></li>
<li><p>Melting snow generates a large amount of runoff around time step 110.</p></li>
<li><p>Melting snow generates a large amount of runoff around time step 130.</p></li>
</ol>
</div></blockquote>